# Spam Detection using Transformers

This notebook demonstrates a basic spam detection system using **BERT** for text classification.

> Note: Spam detection can also be done using simpler ML algorithms like logistic regression or Naive Bayes, which would be faster for small datasets. This example is mainly to understand how **Transformers work** for text classification.


### 1. Install Dependencies

In [3]:
!pip install torch torchvision torchaudio
!pip install transformers datasets scikit-learn

### 2. Load Dataset (SMS Spam Collection)

In [11]:
from datasets import load_dataset

# Load dataset
dataset = load_dataset("sms_spam")
print(dataset)


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/359k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5574 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sms', 'label'],
        num_rows: 5574
    })
})


### 3. Preprocess (Tokenize with BERT)

In [12]:
from transformers import BertTokenizer

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenization function
def tokenize(batch):
    return tokenizer(batch["sms"], padding="max_length", truncation=True, max_length=64)

# Apply tokenization
tokenized_dataset = dataset.map(tokenize, batched=True)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/5574 [00:00<?, ? examples/s]

### 4. Prepare Data for PyTorch

In [13]:
from torch.utils.data import DataLoader

# Set format for PyTorch
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

train_loader = DataLoader(tokenized_dataset["train"], batch_size=16, shuffle=True)


### 5. Load Pretrained BERT for Classification

In [14]:
from transformers import BertForSequenceClassification

# Load pretrained BERT with classification head (2 classes: ham/spam)
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# model = BertForSequenceClassification.from_pretrained("textattack/bert-base-uncased-imdb") - for classification problems

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 6. Training Setup

In [20]:
import torch
from torch.optim import AdamW
from transformers import get_scheduler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Scheduler
num_training_steps = len(train_loader) * 3  # 3 epochs
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

loss_fn = torch.nn.CrossEntropyLoss()


### 7. Training Loop

In [21]:
from tqdm import tqdm

epochs = 3
model.train()

for epoch in range(epochs):
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        batch = {k: v.to(device) for k, v in batch.items()}
        if "label" in batch:
            batch["labels"] = batch.pop("label")

        outputs = model(**batch)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        loop.set_description(f"Epoch {epoch}")
        loop.set_postfix(loss=loss.item())


Epoch 2: 100%|██████████| 349/349 [1:07:41<00:00, 11.64s/it, loss=0.00148]


### Saving the model After Training

In [22]:
output_dir = "./spam_model"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)



('./spam_model/tokenizer_config.json',
 './spam_model/special_tokens_map.json',
 './spam_model/vocab.txt',
 './spam_model/added_tokens.json')

### To save locally

In [24]:
from google.colab import files
import shutil

# zip the folder
shutil.make_archive("spam_model", 'zip', "./spam_model")

# download the zip file
files.download("spam_model.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [28]:
# Reloading the trained Model


from transformers import BertTokenizer, BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("./spam_model")
tokenizer = BertTokenizer.from_pretrained("./spam_model")


### Uploading the model to Hugging Face Hub

In [34]:
!pip install huggingface_hub
from huggingface_hub import notebook_login

notebook_login()


### Push the model

In [36]:
from huggingface_hub import HfApi, HfFolder, Repository, create_repo
from transformers import BertTokenizer, BertForSequenceClassification

# Create a new repo on Hugging Face Hub (only first time)
repo_name = "bert-spam-classifier"   # choose a unique name
create_repo(repo_name, exist_ok=True)

# Push your model to hub
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  /tmp/tmpvu5r2hgx/model.safetensors    :   0%|          | 14.2kB /  438MB            

README.md: 0.00B [00:00, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ssarathi/bert-spam-classifier/commit/02a1cbdd0309734618d529a45b21fd2e5723efe7', commit_message='Upload tokenizer', commit_description='', oid='02a1cbdd0309734618d529a45b21fd2e5723efe7', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ssarathi/bert-spam-classifier', endpoint='https://huggingface.co', repo_type='model', repo_id='ssarathi/bert-spam-classifier'), pr_revision=None, pr_num=None)

### Loading the model anywhere

In [38]:

from transformers import BertTokenizer, BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("ssarathi/bert-spam-classifier")
tokenizer = BertTokenizer.from_pretrained("ssarathi/bert-spam-classifier")


config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

### 8. Evaluation

In [31]:
from sklearn.metrics import accuracy_score

model.eval()
preds, labels = [], []

for batch in train_loader:
    batch = {k: v.to(device) for k, v in batch.items()}

    # Rename 'label' to 'labels' before passing to model
    batch["labels"] = batch.pop("label")

    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)

    preds.extend(predictions.cpu().numpy())
    labels.extend(batch["labels"].cpu().numpy())

print("Accuracy:", accuracy_score(labels, preds))


Accuracy: 0.9996411912450663


### Reloading the model for inference on new set of Emails

In [39]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer

# Device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the saved model and tokenizer
model_path = "ssarathi/bert-spam-classifier"
model = BertForSequenceClassification.from_pretrained(model_path, use_safetensors=True)
tokenizer = BertTokenizer.from_pretrained(model_path)

model.to(device)
model.eval()  # set model to evaluation mode


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

### Tokenize new Emails

In [44]:
new_emails = [
    "Congratulations! You won a free iPhone. Click here to claim.",
    "Hi team, please find the report attached for your review.",
    "Earn $5000 per week from home, no experience required!",
    "Meet me at 10pm tonight in Urban Square"
]


In [45]:
# Tokenize batch of emails
inputs = tokenizer(
    new_emails,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=512
).to(device)


In [46]:
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)

# Convert predictions to labels (adjust based on your training)
label_map = {0: "ham", 1: "spam"}  # assuming 0 = ham, 1 = spam
predicted_labels = [label_map[p.item()] for p in predictions]

# Show results
for email, label in zip(new_emails, predicted_labels):
    print(f"Email: {email}\nPrediction: {label}\n")


Email: Congratulations! You won a free iPhone. Click here to claim.
Prediction: spam

Email: Hi team, please find the report attached for your review.
Prediction: ham

Email: Earn $5000 per week from home, no experience required!
Prediction: spam

Email: Meet me at 10pm tonight in Urban Square
Prediction: ham

